# Project 009 - Transfer Learning with RESNET50 -  Keras Implementation


In [1]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50, resnet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

In [5]:
#(You can swap this out for your own dataset later.)

import os
import cv2
from tensorflow.keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

classes = [
    "airplane", "automobile", "bird", "cat", "deer",
    "dog", "frog", "horse", "ship", "truck"
]

def save_split(split_name, images, labels):
    base = f"cifar10_{split_name}"
    os.makedirs(base, exist_ok=True)

    for i, (img, lbl) in enumerate(zip(images, labels)):
        cls = classes[lbl[0]]
        cls_path = os.path.join(base, cls)
        os.makedirs(cls_path, exist_ok=True)
        cv2.imwrite(os.path.join(cls_path, f"{i}.png"), cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

save_split("train", x_train, y_train)
save_split("test", x_test, y_test)
